In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np


In [10]:
def append_df_to_excel(df, excel_path):
    df_excel = pd.read_excel(excel_path)
    result = pd.concat([df_excel, df], ignore_index=True)
    result.to_excel(excel_path, index=False)

def strip(x):
    if x !=None:
        return   str(x.text.strip())
    else:
        return str('')

In [15]:
empty_list=[]
a = pd.DataFrame(empty_list,columns= ['name','loc','rating','rating_label','number_of_reviews','price','type','stars_count'])

kota = pd.read_csv(r"Indonesian_Family_Life_Survey_4_Longitude_and_Latitude.csv", usecols=[5,6]).drop_duplicates()

cities=kota.iloc[:,1].values.tolist()

print(kota.iloc[101:110,1])


534    JAKARTA SELATAN
544      JAKARTA TIMUR
554      JAKARTA PUSAT
562      JAKARTA BARAT
570      JAKARTA UTARA
576              BOGOR
615           SUKABUMI
660            CIANJUR
690            BANDUNG
Name: kabupaten_name, dtype: object


In [14]:
#url link
my_url='https://www.traveloka.com/en-id/'


#calling web driver from its directory
s=Service(r'C:\Users\62811\Downloads\geckodriver-v0.30.0-win64\geckodriver.exe') 



for j in range(len(kota)):#101,110):#
    try:
        print("current city is", kota.iloc[j,1],j)
        driver = webdriver.Firefox(service=s)
        wait = WebDriverWait(driver, 20)
        driver.get(my_url)
        search_box = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[data-testid='autocomplete-field']"))).send_keys(kota.iloc[j,1])
        time.sleep(3)
        #(wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='autocomplete-item-name' and *contains(string(), %s)]"%city)))).click()
        auto_complete = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@data-testid='autocomplete-item-name'][1]")))

        auto_complete.click()
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-testid='search-submit-button']"))).click()


        x=''
        while True:
            try:
                time.sleep(4)
                x = x + wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "_2qd8A"))).get_attribute('innerHTML') #find_elements_by_class_name("_7192d3184")
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,"next-button"))).click()
                print("Clicked on  Next Page »")
            except TimeoutException:
                print("No more Next Page »")
                break
        driver.quit()
    except TimeoutException:
        driver.quit()
        print("The city:",kota.iloc[j,1],"isn't available")
        continue

    card=BeautifulSoup(x, 'html.parser').find_all(True,{'class':'ztzlF _2nyWi'})

    i=0
    list = [ ]
    for item in card:

        name = strip(item.find(True, {'class':'_1z5je _10ZQX tvat-hotelName'}))
        
        loc = strip(item.find(True,{'class':'_3tICV'}))
        
        rating = strip(item.find(True,{'class':'tvat-ratingScore'}))

        rating_label_ = ''
        rating_label_ = strip(item.find(True,{'class':'_16hDy'})).split()

        if rating_label_ == []:
            rating_label = ''
        else:
            rating_label = rating_label_[0]
        # except:
        #     rating_label = ''

        number_of_reviews = ''
        number_of_reviews = strip(item.find(True,{'class':'_30os4'}))[1:-1]



        price = strip(item.find(True,{'class':'_22n9I tvat-primaryPrice'}))[3:]
        
        type = strip(item.find(True,{'class':'_3ohst Jewfo _2Vswb'}))
        
        # price_for= strip(item.find(True,{'class':'_4abc4c3d5 _7ee1c7d14'}))

        stars = item.find(True,{'class':'_1RoiH _1dIAz tvat-starRating _1Fq-V'})

        # city = kota.iloc[j,1]

        # province_name = kota.iloc[j,0]

        if stars == None:
            stars_count = ''
        elif len(stars) == 1:
            stars_count = ''
        else:
            stars_count=str(len(stars)-1)

        # href_tag= item.find('a', href=True)['href']

        list_in_list = [name,loc,rating,rating_label,number_of_reviews,price,type,stars_count]#,bed,price_for,stars_count,city,href_tag]
        
        list.append(list_in_list)
        #print(list_in_list)
        i=i+1
    df = pd.DataFrame(list,columns= ['name','loc','rating','rating_label','number_of_reviews','price','type','stars_count'])
    df.replace('', np.nan, inplace=True)
    df.dropna(axis=0,how='all')

    #append_df_to_excel(df, r"test_web_scraping_py 1.xlsx")

    import winsound
    duration = 1000  # milliseconds
    freq = 300  # Hz
    winsound.Beep(freq, duration)

current city is Jaksel 101


KeyboardInterrupt: 